In [1]:
# Normal installs
!pip install --upgrade pip
!apt install libgraphviz-dev

  Cloning https://github.com/deepset-ai/haystack.git to c:\users\dell\appdata\local\temp\pip-install-l8j4pes6\farm-haystack_a08ed29509294c9e85622ff3e654cd5d


  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git 'C:\Users\DELL\AppData\Local\Temp\pip-install-l8j4pes6\farm-haystack_a08ed29509294c9e85622ff3e654cd5d'
  error: unable to create file test/samples/crawler/page_with_a_very_long_name_to_do_some_tests_Now_let's_add_some_text_just_to_pass_the_129_chars_mark_and_trigger_the_chars_limit_of_the_default_naming_function.html: Filename too long
  fatal: unable to checkout working tree
  You can inspect what was checked out with 'git status'
  and retry with 'git restore --source=HEAD :/'

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git 'C:\Users\DELL\AppData\Local\Temp\pip-install-l8j4pes6\farm-haystack_a08ed29509294c9e85622ff3e654cd5d' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-w

     ------------------------------------ 118.8/118.8 kB 986.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pygraphviz
Failed to build pygraphviz
  Running setup.py install for pygraphviz: started
  Running setup.py install for pygraphviz: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [46 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.7
      creating build\lib.win-amd64-3.7\pygraphviz
      copying pygraphviz\agraph.py -> build\lib.win-amd64-3.7\pygraphviz
      copying pygraphviz\graphviz.py -> build\lib.win-amd64-3.7\pygraphviz
      copying pygraphviz\scraper.py -> build\lib.win-amd64-3.7\pygraphviz
      copying pygraphviz\testing.py -> build\lib.win-amd64-3.7\pygraphviz
      copying pygraphviz\__init__.py -> build\lib.win-amd64-3.7\pygraphviz
      creating build\lib.win-amd64-3.7\pygraphviz\tests
      copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-3.7\pygraphviz\tests
      copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64-3.7\pygraphviz\tests
      copying pygraphviz\tests\test_close.py -> build\lib.wi

In [ ]:
# install torch scatter
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

In [3]:
# imports
from google.colab import drive
import time
import logging
import os
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes.retriever import EmbeddingRetriever
from haystack.nodes import TableReader
from haystack.utils import print_answers
from haystack import Document
from haystack import Pipeline
import requests
from bs4 import BeautifulSoup as soup
import torch_scatter
import pandas as pd
drive.mount('/content/mnt')
%cd ./mnt/MyDrive/Erasmus/Transformer_Pretraining/

Mounted at /content/mnt
/content/mnt/MyDrive/Erasmus/Transformer_Pretraining


In [ ]:
# configure logger
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# download elasticsearch
%%bash
wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
# start elasticsearch server
%%bash --bg
sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch
time.sleep(30)

In [ ]:
# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_index = "document"
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
import pickle
with open("./data/website_data/processed_website_tables", "rb") as fp:
    processed_website_tables = pickle.load(fp)
with open("./data/website_data/processed_website_text", "rb") as fp:
    processed_website_text = pickle.load(fp)
with open("./data/website_data/processed_schedule_tables", "rb") as fp:
    processed_schedule_tables = pickle.load(fp)

# write website tables to database
document_store.write_documents(processed_website_tables, index=document_index)
# write website text to database
document_store.write_documents(processed_website_text, index=document_index)
# write schedule tables to database
document_store.write_documents(processed_schedule_tables, index=document_index)

In [4]:
# get course list and parse
with open('courses_list.txt', 'r') as my_file:
  courses_list = my_file.read()
courses = pd.read_html(courses_list, flavor="lxml")[0]
courses = courses.rename({courses.columns[0]: 'Name'}, axis=1)
list_soup = soup(courses_list, "html.parser")
list_table = list_soup.find_all('table')[0]
links = []
for tr in list_table.find_all("tr"):
    trs = tr.find_all("td")
    for each in trs:
        try:
            link = each.find('a')['href']
            links.append(link)
        except:
            pass

courses["Link"] = links

In [7]:
# download schedules and filter [function]
def extract_tables(courses):
  courses_contents = []
  processed_tables = []
  for index, row in courses[3:5].iterrows():
    page = requests.get("https://registration.boun.edu.tr/" + row["Link"])
    html_table = soup(page.content, "html.parser").find_all('table')[2]
    course_contents = pd.read_html(str(html_table), flavor="lxml")[0]
    course_contents = course_contents.rename(columns=course_contents.iloc[0]).drop(course_contents.index[0]).reset_index(drop=True)
    course_contents.drop(['Desc.', 'Cr.', 'Quota', 'Course Delivery Method', 'Exam', 'Sl.', 'Required for Dept.(*)', 'Departments'], axis=1, inplace = True)
    course_contents.rename({'Code.Sec': 'Code', 'Instr.': 'Instructor'}, axis=1, inplace = True)
    course_contents.style.set_caption(row["Name"])
    course_contents["Days"] = course_contents["Days"].fillna("No Day specified")
    course_contents["Hours"] = course_contents["Hours"].fillna("No Hours specified")
    course_contents["Rooms"] = course_contents["Rooms"].fillna("No Room specified")
    course_contents["Instructor"] = course_contents["Instructor"].fillna("No Instructor specified")

    courses_contents.append(course_contents)
    document = Document(content=course_contents, content_type="table", id=index)
    processed_tables.append(document)
  return courses_contents, processed_tables

In [8]:
# download schedules and filter
courses_contents, processed_tables = extract_tables(courses)

In [ ]:
# write schedules to database
document_store.write_documents(processed_tables, index=document_index)

In [ ]:
# initialize retriever for tables
retriever = EmbeddingRetriever(document_store=document_store, embedding_model="deepset/all-mpnet-base-v2-table")
# Add table embeddings to the tables in DocumentStore
document_store.update_embeddings(retriever=retriever)

## Retriever TEST
#retrieved_tables = retriever.retrieve("Where is Murat Gülsoy instructor?", top_k=3)

## Get highest scored table
#print(retrieved_tables[0].content)

In [ ]:
# initialize reader
reader = TableReader(model_name_or_path="google/tapas-base-finetuned-wtq", max_seq_len=512)

## Reader TEST
#table_doc = document_store.get_document_by_id(0)
#print(table_doc.content)
#prediction = reader.predict(query="Who teaches readings on asian economics?", documents=[table_doc])
#print_answers(prediction, details="all")
#print(f"Predicted answer: {prediction['answers'][0].answer}")
#print(f"Meta field: {prediction['answers'][0].meta}")

In [ ]:
# Initialize pipeline
table_qa_pipeline = Pipeline()
table_qa_pipeline.add_node(component=retriever, name="EmbeddingRetriever", inputs=["Query"])
table_qa_pipeline.add_node(component=reader, name="TableReader", inputs=["EmbeddingRetriever"])

## Pipeline TEST
#prediction = table_qa_pipeline.run("Who teaches readings on asian economics?", params={"top_k": 2})
#print(prediction['answers'][0].answer)

In [12]:
website_info = pd.read_excel('./urls_website.xlsx')

In [13]:
def extract_texts(website_info):
  table_dfs = []
  website_df = pd.DataFrame(columns = ['Name', 'Content'])
  for index, row in website_info[:13].iterrows():
    if row["Kind"] == "text" and "intl.boun.edu.tr/" not in row["Url"]:
      page = requests.get(row["Url"])
      s = soup(page.content, "html.parser")
      text_passages = [x.getText().replace('\xa0', ' ') for x in s.find(class_='content').find_all('p')]
      website_df = website_df.append({'Name' : row["Topic"], 'Content' : ("\n").join(text_passages)}, ignore_index = True)
    elif row["Kind"] == "table" or "intl.boun.edu.tr/" in row["Url"]:
      page = requests.get(row["Url"])
      text_tables = []
      if "intl.boun.edu.tr/" in row["Url"]:
        html_tables = soup(page.content, "html.parser").find(class_='region-content').find_all('table')
      elif "boun.edu.tr/" in row["Url"]:
        html_tables = soup(page.content, "html.parser").find(class_='content').find_all('table')
      #print(f"# tables: {len(html_tables)}")
      for html_table in html_tables:
        if len(html_table.findAll(lambda tag: tag.name == 'tr')) == 1:
          text_passage = html_table.getText().replace('\xa0', ' ')
          text_tables.append(text_passage)

        elif len(html_table.findAll(lambda tag: tag.name == 'tr')) > 1: 
          table_df = pd.read_html(str(html_table), flavor="lxml")[0]
          table_df = table_df.rename(columns=table_df.iloc[0]).drop(table_df.index[0]).reset_index(drop=True)
          table_dfs.append(table_df)
      if len(text_tables) > 0:
        website_df.append({'Name' : row["Topic"], 'Content' : ("\n").join(text_tables)}, ignore_index = True)
  return website_df, table_dfs

In [14]:
w_df, t_dfs = extract_texts(website_info)

In [16]:
w_df

,Name,Content
0,Campuses,Many of the University's buildings are located...
1,South Campus,South Campus is the historical campus of Bogaz...
2,North Campus,Abdullah Kuran Library is the main library of ...
3,Hisar Campus,Overlooking the Bosphorus and the Fatih Sultan...
4,Ucaksavar Campus,Ucaksavar Campus has housing facilities for th...
5,Kandilli Campus,This campus is home to Kandilli Observatory wh...
6,Santepe Campus,Sarıtepe Campus which is situated on the shore...
7,History of Bogazici University,"Robert College was founded in Istanbul,Turkey,..."
8,Vision Mission,Boğaziçi University’s vision is to be a leadin...


In [ ]:
website_df.iloc[1]["Content"]

'South Campus is the historical campus of Bogazici University situated on a hill overlooking the Bosphorous and the fortress, Rumeli Hisari. It houses Albert Long Hall, the main cultural center of the university, Dodge Gymnasium, Henrietta Washburn Hall which serves as a student activity center with a theater hall, the old Van Millingen Library which now houses the Rector’s office and the Gates Hall that has the Registrar’s Office and the Office of Financial Affairs.\nThe academic departments in South Campus are housed in Washburn Hall (Political Science and Business Administration Departments of the Faculty of Economic and Administrative Science FEAS), Anderson Hall (Faculty of Arts and Sciences-FAS), Perkins Hall (Civil, Industrial and Mechanical Engineering Departments of the Faculty of Engineering), Sloane Hall (Psychology and Sociology Departments of FAS), Natuk Birkan Building (Economics Department of FEAS and Turkish Language and Literature Department of FAS) and the School of F